In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

questions = pd.read_csv('https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Questions.csv',engine='python',encoding='ISO-8859-1')
answers = pd.read_csv('https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Answers.csv',engine='python',encoding='ISO-8859-1')
tags = pd.read_csv('https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Tags.csv',engine='python',encoding='ISO-8859-1')
users = pd.read_csv('https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Users.csv',engine='python',encoding='ISO-8859-1')
print('All the stack overflow data set loaded')

All the stack overflow data set loaded


In [2]:
#Elimination#1 - perform EDA on question dataset
questions = questions[~((questions.OwnerUserId.isnull()) & (questions.ClosedDate.isnull()) & (questions.Score ==0))]
questions = questions[~((questions.ClosedDate.isnull()) & (questions.Score ==0))]
questions = questions[~((questions.OwnerUserId.isnull()) & (questions.Score ==0))]
questions = questions[~(questions.OwnerUserId.isnull())]
questions.isnull().sum()/len(questions)*100

Id               0.000000
OwnerUserId      0.000000
CreationDate     0.000000
ClosedDate      91.933022
Score            0.000000
Title            0.000000
Body             0.000000
dtype: float64

In [3]:
#Elimination#2 - perform EDA on answers dataset
answers = answers[~((answers.OwnerUserId.isnull()) & (answers.Score ==0))]
answers = answers[~(answers.OwnerUserId.isnull())]
answers.isnull().sum()/len(answers)*100

Id              0.0
OwnerUserId     0.0
CreationDate    0.0
ParentId        0.0
Score           0.0
Body            0.0
dtype: float64

In [4]:
#Elimination#3 - perform EDA on tags dataset
tags = tags[~tags.Tag.isnull()]
tags.drop_duplicates(subset='Tag',keep='first',inplace=True)
tags.isnull().sum()/len(tags)*100

Id     0.0
Tag    0.0
dtype: float64

In [5]:
#Elimination#4 - perform EDA on users dataset
users[(users.up_votes ==0) & (users.down_votes ==0) & (users.views == 0)]
del users['Unnamed: 0']
users.drop_duplicates(keep='first',inplace=True)
users = users[~(users.display_name.isnull())]
users.isnull().sum()/len(users)*100

id                  0.0
display_name        0.0
creation_date       0.0
last_access_date    0.0
reputation          0.0
up_votes            0.0
down_votes          0.0
views               0.0
dtype: float64

In [0]:
def getUserNameFromQuestion(questionId):
  username = ''
  try:
    userIdFromQuestion = questions[questions['Id'] == questionId].OwnerUserId
    username = users[users['id'].values == userIdFromQuestion.values].display_name.values
  except Exception:
    print('Error occured when retrieving user name for the question '+questionId)
  finally:
    return username



In [10]:
#When a user raises a question with a data science tag ('python', 'r', 'matlab', 'sas', 'excel', 'sql'), 
#then Stack Overflow sends the notification to the relevant users.
def sendQuestionNotification():
  questionTagList = ['python', 'r', 'matlab', 'sas', 'excel', 'sql']

  #Iterate tagged question
  for qTag in questionTagList:  
    taggedQuestionList = tags[tags['Tag'].str.contains(qTag)].Id.values.tolist()

    for questionId in taggedQuestionList:
      username = getUserNameFromQuestion(questionId)
      print('Notification sent to user '+username+' who added the tag '+ qTag+' to the question '+str(questionId))
      break #comment the break to see the full listing of notification.  Don't do that until you want to see the full notification listing

sendQuestionNotification()

['Notification sent to user jkp who added the tag python to the question 11060']
['Notification sent to user Shawn who added the tag r to the question 80']
['Notification sent to user mana who added the tag matlab to the question 83190']
['Notification sent to user Frank V who added the tag sas to the question 102600']
[]
['Notification sent to user Zack Peterson who added the tag sql to the question 120']


In [11]:

#When a user provides a data science tag ('python', 'r', 'matlab', 'sas', 'excel', 'sql') to their question, 
#then Stack Overflow should come up with the expected approximate time to get it answered.
def getApproxAnswerTime():
  dataScienceTagList = ['python', 'r', 'matlab', 'sas', 'excel', 'sql']  

  #Iterate data science tagged question
  for qTag in dataScienceTagList:  
    #get all the question ids tagged against specific value
    taggedQuestionList = tags[tags['Tag'].str.contains(qTag)].Id.values.tolist()
    #get all the answer id tagged against specific value
    answerDates = answers[answers['ParentId'].isin(taggedQuestionList)]
    answerDates['CreationDate'] = pd.to_datetime(answerDates['CreationDate'])
    avgResponseDays = answerDates.CreationDate.dt.day.mean()
    print('Average Response Time for '+qTag +' tagged question is', avgResponseDays, ' days')



getApproxAnswerTime()

Average Response Time for python tagged question is 16.27076923076923  days
Average Response Time for r tagged question is 15.755619007701577  days
Average Response Time for matlab tagged question is 16.609756097560975  days
Average Response Time for sas tagged question is 18.53030303030303  days
Average Response Time for excel tagged question is 18.3  days
Average Response Time for sql tagged question is 16.099775784753362  days


In [12]:
#While a user raises a question, Stack Overflow wants to provide some suggestions to the users to get their queries answered quickly.
def getSuggestedAnswers():

    #Take sampe 10 tags for demonstration purpose
    tagsList = tags.Tag.head(10).values.tolist()
    
    for qTag in tagsList: 
      #get all the question ids tagged against specific value
      taggedQuestionList = tags[tags['Tag'].str.contains(qTag)].Id.values.tolist()
      #print(taggedQuestionList)
    
      #pickup top 5 suggested answers
      suggestedAnswers = answers[answers['ParentId'].isin(taggedQuestionList)].Body#.head(5).values.tolist()
      #print(suggestedAnswers[:5])
      print('Top 5 Suggested Answers List for the tag '+qTag+ 'is ')
      print('=====================================================')

      answerid = 1
      for anAnswer in suggestedAnswers[:5]:
        print('Answer'+str(answerid)+'.', anAnswer)
        answerid += 1

getSuggestedAnswers()

Top 5 Suggested Answers List for the tag flexis 
Answer1. <p>I wound up using this. It is a kind of a hack, but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D</p>

<pre><code>var strSql:String = stream.readUTFBytes(stream.bytesAvailable);      
var i:Number = 0;
var strSqlSplit:Array = strSql.split(";");
for (i = 0; i &lt; strSqlSplit.length; i++){
    NonQuery(strSqlSplit[i].toString());
}
</code></pre>

Answer2. <p>The <a href="http://en.wikipedia.org/wiki/SQLite" rel="nofollow">SQLite</a> API has a function called something like <code>sqlite_prepare</code> which takes <em>one</em> statement and prepares it for execution, essentially parsing the SQL and storing it in memory. This means that the SQL only has to be sent once to the database engine even though the statement is executed many times.</p>

<p>Anyway, a statement is a single SQL query, that's just the rule. The AIR SQL API doesn't allow sending raw SQL to SQLite, only s

In [69]:
#Let’s say Stack Overflow also wants to post some details on its job website to help various employees understand 
#the subjects or topics trending in the industry.
def getTop5TrendingTopics():

  grouped = tags.groupby(['Id'])['Tag'].count().reset_index(name='count').sort_values(['count'], ascending=False)
  #print(grouped[1:6])
  for name,group in grouped[1:6].iterrows():
    print('Top 5 Trending Topic=',tags[tags.Id == group.Id].Tag.values.tolist())      


getTop5TrendingTopics()

Top 5 Trending Topic= ['mathematical-optimization', 'linear-programming', 'cplex', 'gurobi', 'convex-optimization']
Top 5 Trending Topic= ['css', 'validation', 'xhtml', 'cross-browser', 'w3c']
Top 5 Trending Topic= ['rdf', 'semantics', 'semantic-web', 'owl', 'ontology']
Top 5 Trending Topic= ['iis-6', 'windows-server-2003', 'cluster-computing', 'failover']
Top 5 Trending Topic= ['listview', 'header', 'themes', 'glyph']


In [67]:
#grouped = tags.groupby(['Id','Tag'],).count()
grouped = tags.groupby(['Id'])['Tag'].count().reset_index(name='count').sort_values(['count'], ascending=False)
print(grouped[1:5])
for name,group in grouped[1:5].iterrows():
  print('Id=',group.Id,'Trending Topic=',tags[tags.Id == group.Id].Tag.values.tolist())

         Id  count
952  143020      5
67     7940      5
508   66720      5
170   22590      4
Id= 143020 Trending Topic= ['mathematical-optimization', 'linear-programming', 'cplex', 'gurobi', 'convex-optimization']
Id= 7940 Trending Topic= ['css', 'validation', 'xhtml', 'cross-browser', 'w3c']
Id= 66720 Trending Topic= ['rdf', 'semantics', 'semantic-web', 'owl', 'ontology']
Id= 22590 Trending Topic= ['iis-6', 'windows-server-2003', 'cluster-computing', 'failover']


In [66]:
tags[tags.Id == 143020]

,Id,Tag
4881,143020,mathematical-optimization
4882,143020,linear-programming
4883,143020,cplex
4884,143020,gurobi
4885,143020,convex-optimization
